In [6]:
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree, NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from pysparnn.cluster_index import MultiClusterIndex

In [2]:
vectors = pickle.load(open('vectors.pkl','rb'))

In [8]:
data = pd.read_csv('data.csv')

In [9]:
index = MultiClusterIndex(vectors, data)

KeyboardInterrupt: 

In [ ]:
with open('nn_index.pkl', 'wb') as f:
    pickle.dump(index, f)

In [15]:
tree = NearestNeighbors(n_neighbors=3, algorithm='kd_tree', leaf_size=100)
tree.fit(vectors)

/data/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='kd_tree', leaf_size=100, n_neighbors=3)

In [12]:
tree.kneighbors(vectors[4], n_neighbors=1)

(array([[0.]]), array([[4]]))